# RAG with Multi-Agent Debate

Inspired by [More Agent Is All You Need from Li et al. 2024](https://arxiv.org/pdf/2402.05120.pdf)

In [4]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate

command_r_plus = dspy.Cohere(model="command-r-plus", max_tokens=4000, api_key=cohere_api_key)
command_r = dspy.Cohere(model="command-r", temperature=0.8, max_tokens=4000, api_key=cohere_api_key)
weaviate_client = weaviate.connect_to_local()
weaviate_rm = WeaviateRM("WeaviateBlogChunk",
                        weaviate_client=weaviate_client)
dspy.settings.configure(lm=command_r, rm=weaviate_rm)

# Demonstration of Output Sampling

In [6]:
for i in range(5):
    print(f"\033[91mGeneration {i}:\n")
    response = command_r("Can you please write a sentence that uses all the letters in the alphabet")[0]
    print(f"\033[0m{response}\n")

Generation 0:

From aforethought, we can discern the first letter of the alphabet, A, through the last, Z, in this sentence.

Generation 1:

From aardvarks to zombies, the alphabet soup is swirling with quirky creatures.

Generation 2:

From auntie's zealous zest, the jolly kids played hide-and-seek in the lively meadow, enjoying every moment.

Generation 3:

From adeptness to zesty, the multilingual capability encompasses the entire alphabet.

Generation 4:

From airy Alaska to zoetic Zanzibar, the world offers an omnishambles of adventures to explore, encompassing all the letters of the alphabet.



# Signatures for `RAGwithMultiAgentDebate`

In [7]:
class GenerateAnswer(dspy.Signature):
    """Assess the context and write a detailed 3-4 sentence answer to the question."""
    
    contexts = dspy.InputField()
    question = dspy.InputField()
    detailed_answer = dspy.OutputField(desc="Only output the answer")
    
class ResolveCandidateAnswers(dspy.Signature):
    """
    You are a moderator for question-answering.
You will receive a few candidate answers to a question, as well as some additional context about the topic.
Please resolve the candidate answers into a single answer integrating the strengths of each candidate answer.
    """
    
    contexts = dspy.InputField()
    candidate_answers = dspy.InputField()
    resolved_answer = dspy.OutputField(desc="Only output the resolved_answer")

# `RAGwithMultiAgentDebate` program

In [8]:
from typing import List
def format_candidate_answers(candidate_responses: List[str]) -> str:
    formatted_string = ""
    for i, string in enumerate(candidate_responses, start=1):
        formatted_string += f"[{i}] {string} "
    return formatted_string.strip()

class RAGwithMultiAgentDebate(dspy.Module):
    def __init__(self):
        self.retrieve = dspy.Retrieve()
        self.answer_question = dspy.Predict(GenerateAnswer)
        self.resolve_candidate_answers = dspy.Predict(ResolveCandidateAnswers)

    def forward(self, question, candidate_generations=5):
        contexts = self.retrieve(question).passages
        contexts = "".join(contexts)
        candidate_answers = []
        for i in range(candidate_generations):
            with dspy.context(lm=command_r):
                candidate_answers.append(self.answer_question(contexts=contexts, question=question).detailed_answer)
        candidate_answers = format_candidate_answers(candidate_answers)
        with dspy.context(lm=command_r_plus):
            final_answer = self.resolve_candidate_answers(candidate_answers=candidate_answers, contexts=contexts).resolved_answer
        return dspy.Prediction(answer=final_answer)

# Test!

In [10]:
print(RAGwithMultiAgentDebate()(question="What is ref2vec?").answer)

Ref2Vec, short for reference-to-vector, is a Weaviate module that enables vectorization of data objects by leveraging their cross-references to other objects. It calculates the average, or centroid vector, of the cross-referenced vectors to represent the referencing object, finding a middle ground between all related objects. This lightweight and efficient method can be used for various tasks, including recommendations, re-ranking, and representing lengthy objects in a more concise and relevant manner. Ref2Vec, also known as ref2vec-centroid, enhances recommendation systems by updating user vectors in real-time based on their preferences and actions, resulting in fast and personalized recommendations.


# Observe the Final Prompt to Command R+ 

In [11]:
command_r_plus.inspect_history(n=1)




You are a moderator for question-answering.
You will receive a few candidate answers to a question, as well as some additional context about the topic.
Please resolve the candidate answers into a single answer integrating the strengths of each candidate answer.

---

Follow the following format.

Contexts: ${contexts}
Candidate Answers: ${candidate_answers}
Resolved Answer: Only output the resolved_answer

---

Contexts: --- title: What is Ref2Vec and why you need it for your recommendation system slug: ref2vec-centroid authors: [connor] date: 2022-11-23 tags: ['integrations', 'concepts'] image: ./img/hero.png description: "Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!" --- ![Ref2vec-centroid](./img/hero.png) <!-- truncate --> Weaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectorizer-modules/ref2vec-centroid) module. In this article, we give you an overview of what Ref2Vec is and some examples in which it 

'\n\n\nYou are a moderator for question-answering.\nYou will receive a few candidate answers to a question, as well as some additional context about the topic.\nPlease resolve the candidate answers into a single answer integrating the strengths of each candidate answer.\n\n---\n\nFollow the following format.\n\nContexts: ${contexts}\nCandidate Answers: ${candidate_answers}\nResolved Answer: Only output the resolved_answer\n\n---\n\nContexts: --- title: What is Ref2Vec and why you need it for your recommendation system slug: ref2vec-centroid authors: [connor] date: 2022-11-23 tags: [\'integrations\', \'concepts\'] image: ./img/hero.png description: "Weaviate introduces Ref2Vec, a new module that utilises Cross-References for Recommendation!" --- ![Ref2vec-centroid](./img/hero.png) <!-- truncate --> Weaviate 1.16 introduced the [Ref2Vec](/developers/weaviate/modules/retriever-vectorizer-modules/ref2vec-centroid) module. In this article, we give you an overview of what Ref2Vec is and some